#### enumerate함수
많은 양의 데이터인 list를 다룰때 편리하게 쓸수있는 기능을 제공

## for-each , for..
#### for x in list
리스트에 있는 것을 하나씩 꺼내서 처리하기가 아주 간단하나 index가 없어서 불편. 
하나씩 꺼내서 출력, 연산하기에는 용이하다. 리스트에 입력은 안된다...

#### for i in range(n)
리스트의 입력과 출력이 다되나 인덱스로 리스트에 있는 값들에 접근해야하기때문에 코드가 복잡해보임. 


In [115]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [116]:
num_list=[100,200,300]

In [117]:
for x in num_list:
    print(x+100)
    
#출력용으로 용이하나, 리스트의 인덱스의 값을 수정하기는 어렵다.

200
300
400


In [118]:
for i, x in enumerate(num_list):
    print(i,':',x+100)
    num_list[i]=x+100

0 : 200
1 : 300
2 : 400


In [119]:
 num_list

[200, 300, 400]

In [120]:
from sklearn.model_selection import KFold

In [121]:
kfold=KFold(n_splits=5)

In [122]:
# titanic=pd.read_excel("../csv-data/titanic.xls")
titanic=pd.read_csv("../csv-data/titanic_train.csv")
titanic.head(2)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,body,home.dest
0,2,1,"Mellinger, Miss. Madeleine Violet",female,13.0,0,1,250644,19.5,NaN,S,NaN,"England / Bennington, VT"
1,2,1,"Wells, Miss. Joan",female,4.0,1,1,29103,23.0,NaN,S,NaN,"Cornwall / Akron, OH"


In [123]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899 entries, 0 to 898
Data columns (total 13 columns):
pclass       899 non-null int64
survived     899 non-null int64
name         899 non-null object
sex          899 non-null object
age          729 non-null float64
sibsp        899 non-null int64
parch        899 non-null int64
ticket       899 non-null object
fare         899 non-null float64
cabin        211 non-null object
embarked     897 non-null object
body         84 non-null float64
home.dest    516 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 91.4+ KB


In [124]:
def fill_na(df):
    df.isnull().sum() 
    df['age']=df['age'].fillna(df['age'].mean())
    df['cabin']=df['cabin'].fillna('N')
    df['embarked']=df['embarked'].fillna('N')
    df['cabin']=df['cabin'].str[:1]
    return df

def drop_col(df):
    df.drop(columns=['name','ticket','body','home.dest'], inplace=True)
    return df

def label_encoding(df):
    le=LabelEncoder()
    features=['sex','cabin','embarked']
    for f in features:
        le.fit(df[f].unique())
        print("라벨링한 목록:", le.classes_)
        df[f]=le.transform(df[f])
    return df

In [125]:
def preprocessing(df):
    df=fill_na(df)
    df=drop_col(df)
    df=label_encoding(df)
    return df
#전처리 담당하는 함수. 

In [126]:
titanic2=titanic.copy()

In [127]:
titanic2=preprocessing(titanic2)

라벨링한 목록: ['female' 'male']
라벨링한 목록: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'N' 'T']
라벨링한 목록: ['C' 'N' 'Q' 'S']


In [128]:
titanic3=titanic2[['pclass','sex', 'age', 'sibsp', 'parch', 'fare', 'cabin',
       'embarked']]
titanic3

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
0,2,0,13.000000,0,1,19.5000,7,3
1,2,0,4.000000,1,1,23.0000,7,3
2,2,0,30.000000,1,0,13.8583,7,0
3,3,1,30.188381,0,0,7.7250,7,2
4,3,0,22.000000,0,0,7.7250,7,2
...,...,...,...,...,...,...,...,...
894,3,1,30.188381,0,0,7.8792,7,3
895,1,1,64.000000,1,4,263.0000,2,3
896,3,1,19.000000,0,0,8.0500,7,3
897,1,0,26.000000,1,0,136.7792,2,0


In [170]:
X_train, X_test, y_train, y_test=train_test_split(titanic3, titanic2['survived'], test_size=0.2, random_state=100)

In [171]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((719, 8), (180, 8), (719,), (180,))

#### 8개의 변수로 1개의 target을 분류하는 문제
##### y = a * x1 + b * x2 + c * x3 + d * x4 + e * x5 + f * x6 + g * x7 + h * x8 +B
x=['pclass','sex', 'age', 'sibsp', 'parch', 'fare', 'cabin','embarked']

....

survived=a * pclass + b * sex + c * age + d * sibsp + e * parch + f * fare + g * cabin + h * embarked +B

In [172]:
df_clf=DecisionTreeClassifier(random_state=11)
rf_clf=RandomForestClassifier(random_state=11)
svc_clf=SVC()

## DecisionTreeClassifier

In [173]:
df_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=11, splitter='best')

In [174]:
df_clf.score(X_train, y_train)

0.9791376912378303

In [175]:
com_target=df_clf.predict(X_test)

In [176]:
accuracy_score(y_test, com_target)

0.7944444444444444

## RandomForestClassifier

In [177]:
rf_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=11, verbose=0,
                       warm_start=False)

In [178]:
rf_clf.score(X_train, y_train)

0.96105702364395

In [179]:
com_target=rf_clf.predict(X_test)
accuracy_score(y_test, com_target)

0.7944444444444444

## SVC

In [180]:
svc_clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [181]:
svc_clf.score(X_train, y_train)

0.8970792767732962

In [182]:
com_target=svc_clf.predict(X_test)
accuracy_score(y_test, com_target)

0.7166666666666667

## for문 이용해서... 비교!
반복되는 부분이 많아서


In [183]:
how=[df_clf, rf_clf, svc_clf]


In [185]:
for h in how:
    h.fit(X_train, y_train)
    com_target=h.predict(X_test)
    print("정확도", accuracy_score(y_test, com_target))

정확도 0.7944444444444444
정확도 0.7944444444444444
정확도 0.7166666666666667


## 교차 검증....

In [186]:
kfold=KFold(n_splits=5)

In [192]:
for i, (train_index, test_index) in enumerate(kfold.split(titanic3)):
    print (i,"-----------")
    print("test_index>>" , test_index )
    print("train_index>>" , train_index )
# x= (train_idex,test_index)

0 -----------
test_index>> [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179]
train_index>> [180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 

In [198]:
X_test = titanic3.values[test_index]
X_test = titanic3.values[test_index]

array([[  3.        ,   1.        ,  18.5       , ...,   7.2292    ,
          7.        ,   0.        ],
       [  1.        ,   0.        ,  30.18838134, ...,  79.2       ,
          7.        ,   0.        ],
       [  1.        ,   0.        ,  22.        , ..., 151.55      ,
          7.        ,   3.        ],
       ...,
       [  3.        ,   1.        ,  19.        , ...,   8.05      ,
          7.        ,   3.        ],
       [  1.        ,   0.        ,  26.        , ..., 136.7792    ,
          2.        ,   0.        ],
       [  3.        ,   0.        ,  24.        , ...,  15.85      ,
          7.        ,   3.        ]])

In [249]:
def fold(nsplit, clf):
    acc=[]
    kfold=KFold(n_splits=nsplit)
    for i, (train_index, test_index) in enumerate(kfold.split(titanic3)):
        X_train= titanic3.values[train_index]
        y_train= titanic2['survived'].values[train_index]
        X_test= titanic3.values[test_index]
        y_test= titanic2['survived'].values[test_index]
        clf.fit(X_train, y_train)
        com_target=clf.predict(X_test)
        acc.append(accuracy_score(y_test, com_target))
        print("정확도", accuracy_score(y_test, com_target))
    print("평균 정확도 : ", np.mean(acc))
    return np.mean(acc)

#### 함수 설명 def fold(nsplit, clf)
# nsplit |  kFold에 줄 n_splits의 값!
# clf    |  학습 돌릴 모델이름! 


### k-fold로 나눈 데이터 셋으로 교차검증하기~

In [250]:
how=[df_clf, rf_clf, svc_clf]
mean=[]

for h in how:
    a=str(h)
    print("-------------------------------------"+a[: a.find("(")]+"---------------------------------")
    mean.append(fold(10,h))
print("------------------------------------------------------------------------------------------------------------------")
gg=str(how[mean.index(max(mean))])
print("제일 정확도가 높은 모델은",gg[: gg.find("(")],"이고, 평균 정확도는",max(mean), "입니다." )
print("------------------------------------------------------------------------------------------------------------------")

-------------------------------------DecisionTreeClassifier---------------------------------
정확도 0.8222222222222222
정확도 0.7333333333333333
정확도 0.7111111111111111
정확도 0.7777777777777778
정확도 0.8
정확도 0.7444444444444445
정확도 0.7333333333333333
정확도 0.8222222222222222
정확도 0.8333333333333334
정확도 0.7528089887640449
평균 정확도 :  0.7730586766541822
-------------------------------------RandomForestClassifier---------------------------------
정확도 0.8444444444444444
정확도 0.7888888888888889
정확도 0.8
정확도 0.7333333333333333
정확도 0.8
정확도 0.7333333333333333
정확도 0.7333333333333333
정확도 0.8222222222222222
정확도 0.8111111111111111
정확도 0.797752808988764
평균 정확도 :  0.786441947565543
-------------------------------------SVC---------------------------------
정확도 0.7111111111111111
정확도 0.6888888888888889
정확도 0.6333333333333333
정확도 0.6222222222222222
정확도 0.6444444444444445
정확도 0.5777777777777777
정확도 0.7555555555555555
정확도 0.6666666666666666
정확도 0.7
정확도 0.6741573033707865
평균 정확도 :  0.6674157303370787
-------------------------